In [ ]:
# install quantum espresso from conda
# !conda install conda-forge::qe
# !pip install ase

In [10]:
from ase.io import read
from ase.calculators.espresso import Espresso, EspressoProfile

atoms = read("Mn0.25Zn0.75(FeO2)2.cif")  # Replace with your actual file

pseudos = {
    "Fe": "Fe.pbe-spn-kjpaw_psl.0.2.1.UPF",
    "Mn": "mn_pbe_v1.5.uspp.F.UPF",
    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
    "Zn": "Zn_pbe_v1.uspp.F.UPF",
}

profile = EspressoProfile(command="/opt/conda/bin/pw.x", pseudo_dir="./pseudo")

input_data = {
    "control": {
        "calculation": "scf",
        "etot_conv_thr": 5.6e-3,
        "forc_conv_thr": 1e-3,
        "outdir": "./out/",
        "prefix": "aiida",
        "pseudo_dir": "./pseudo/",
        "tprnfor": True,
        "tstress": True,
        "verbosity": "high",
    },
    "system": {
        "degauss": 0.0275,
        "ecutrho": 1080,
        "ecutwfc": 90,
        "ibrav": 0,
        "nat": 56,
        "nosym": False,
        "nspin": 2,
        "ntyp": 4,
        "occupations": "smearing",
        "smearing": "cold",
        "starting_magnetization": {
            "Fe": 0.3125,
            "Mn": 0.3333,
            "O": 0.1,
            "Zn": 0.1,
        },
    },
    "electrons": {
        "conv_thr": 2.24e-8,
        "electron_maxstep": 80,
        "mixing_beta": 0.4,
    },
}

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudos,
    kpts=(1, 1, 1),
    input_data=input_data,
    cell=[
        [8.1792, 0.0, 0.0],
        [0.0, 8.1792, 0.0],
        [0.0, 0.0, 8.1792],
    ],
)

atoms.calc = calc
energy = atoms.get_potential_energy()
print("Total energy (eV):", energy)


CalledProcessError: Command '['/opt/conda/bin/pw.x', '-in', 'espresso.pwi']' returned non-zero exit status 1.

### Update calculator for Band calculation

In [ ]:
fermi_level = calc.get_fermi_level()

In [ ]:
# Get the lattice and band path
lattice = struct.get_cell()
bandpath = lattice.bandpath(npoints=50)
input_data['control'].update({'calculation':'bands','restart_mode':'restart','verbosity':'high'})
# Set up the calculator for band structure calculation

In [ ]:
struct.calc = Espresso(profile=profile,
    pseudopotentials=pseudo_dict, kpts={'path': bandpath.kpts},
    input_data=input_data)


### Start Band calculation

In [ ]:
struct.calc.calculate(struct, [], [])

In [ ]:
from ase.spectrum.band_structure import BandStructure, get_band_structure
bands = get_band_structure(struct, reference=fermi_level)

In [ ]:
bands.plot(show=True, emin=fermi_level - 5, emax=fermi_level + 5.0)

In [ ]:
! ls